2021021699 Yuwon Lee<br>
2021021581 Jinha Lim<br>
2021021580 Shinhye Lee

In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

Tried to Stitch images one by one, the quality significantly droped after linking more than 4 images

In [82]:
path = '/home/prml/workspace/ComputerVision/ComputerVision/Assignment4'
bf = cv2.BFMatcher()

#Loading images from directory
img_1 = cv2.imread('/home/prml/workspace/ComputerVision/ComputerVision/Assignment4/output09-12.png')
img1 = cv2.cvtColor(img_1,cv2.COLOR_BGR2GRAY)

img_2 = cv2.imread('/home/prml/workspace/ComputerVision/ComputerVision/Assignment4/output13-18.png')
img2 = cv2.cvtColor(img_2,cv2.COLOR_BGR2GRAY)

orb = cv2.ORB_create(
    nfeatures=10000,
    scaleFactor=1.2,
    nlevels=8,
    edgeThreshold=31,
    firstLevel=0,
    WTA_K=2,
    scoreType=cv2.ORB_HARRIS_SCORE,
    patchSize=31,
    fastThreshold=20,
)

kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)
matches = bf.knnMatch(des1,des2, k=2)

good = []
for m in matches:
    if (m[0].distance < 0.7*m[1].distance):
        good.append(m)
matches = np.asarray(good)

if (len(matches[:,0]) >= 5):
    src = np.float32([ kp1[m.queryIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
    dst = np.float32([ kp2[m.trainIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
    H, masked = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)

    dst = cv2.warpPerspective(img_1,H,((img_1.shape[1] + img_2.shape[1]), img_2.shape[0])) #wraped image
    dst[0:img_2.shape[0], 0:img_2.shape[1]] = img_2 #stitched image
    
    #Cropping blank part of the stitched image
    dst_gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(dst_gray, 0, 255, cv2.THRESH_OTSU)
    
    bbox = cv2.boundingRect(thresholded)
    x,y,w,h = bbox
    
    dst = dst[y:y+h, x:x+w]

    cv2.imwrite('output09-18.png',dst)

Tried Panorama function to automatically stitch all images

In [ ]:
path = '/home/prml/workspace/ComputerVision/ComputerVision/Assignment4/images'
images=[]
img_input=[]

#loading images from directory
for root, directories, files in os.walk(path):
    for file in files:
        if '.png' in file:
            img_input=cv2.imread(os.path.join(root,file))
            images.append(img_input)

#Used CV2 stitcher's panorama function
stitcher = cv2.Stitcher.create(cv2.Stitcher_PANORAMA)
status, pano = stitcher.stitch(images)

if status != cv2.Stitcher_OK:
    print("error")
    exit(-1)

cv2.imwrite('output.jpg',pano)